In [ ]:
%pip show kubeflow-training  # to check installed kubeflow-traning SDK version during test-run

In [ ]:
import sys
sys.path.append("../notebooks")  # needed to make kfto_sdk_mnist
from kfto_sdk_mnist import train_func 

In [ ]:
# parameters
num_gpus = "${num_gpus}"
openshift_api_url = "${api_url}"
namespace = "${namespace}"
token = "${token}"
training_image= "${training_image}"
localQueue = "${localQueue}"

In [ ]:
from kubernetes import client
from kubeflow.training import TrainingClient

configuration = client.Configuration()
configuration.host = openshift_api_url
configuration.api_key = {"authorization": f"Bearer {token}"}
configuration.verify_ssl = False
api_client = client.ApiClient(configuration)
client = TrainingClient(client_configuration=api_client.configuration)

In [ ]:
import os
client.create_job(
   name="pytorch-ddp",
   namespace=namespace,
   train_func=train_func,
   num_workers=1,
   resources_per_worker={"gpu": num_gpus},
   base_image=training_image,
   packages_to_install=["torchvision==0.19.0","minio==7.2.13"],
   pip_index_url= os.environ.get("PIP_INDEX_URL"),
   env_vars={
      "NCCL_DEBUG": "INFO", 
      "TORCH_DISTRIBUTED_DEBUG": "DETAIL", 
      "DEFAULT_PIP_INDEX_URL": os.environ.get("PIP_INDEX_URL"),
      "PIP_TRUSTED_HOST": os.environ.get("PIP_TRUSTED_HOST")
   },
   labels={
       "kueue.x-k8s.io/queue-name": localQueue,
   }
)

In [ ]:
import time
while not client.is_job_succeeded(name="pytorch-ddp", namespace=namespace): 
    time.sleep(1)
print("PytorchJob Succeeded!")

In [ ]:
time.sleep(10)